# MNIST Embeddings

## Imports and Hyper-parameters 

In [338]:
import torch 
import os 
import numpy as np 
from torchvision import transforms 
from torchvision.datasets import MNIST 
from torchvision import models 

print(os.getcwd())
root = os.getcwd()

from DeepFeatures import DeepFeatures
from mnist_net import Net

from sklearn.ensemble import IsolationForest
import plotly.express as px
import pandas as pd

/home/praveens/Desktop/synthetic_biometrics/visualize_embeddings


In [339]:
batch_size = 128
data_folder = root + '/MNIST'
device = 'cpu'
model_path = './mnist_net.pth'

merge_class1 = 2
merge_class2 = 4

## Create Dataloader

In [340]:
transformations = transforms.Compose([transforms.Resize((221, 221)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485], std=[0.229])
                                    ])

mnist_data = MNIST(root=r'./MNIST',
                   download=False, # change to True to download MNIST data
                   train=False, 
                   transform=transformations)

data_loader = torch.utils.data.DataLoader(mnist_data,
                                          batch_size=batch_size,
                                          shuffle=True)

In [341]:
mnist_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./MNIST
    Split: Test
    StandardTransform
Transform: Compose(
               Resize(size=(221, 221), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485], std=[0.229])
           )

In [342]:
# list(mnist_net.named_parameters())

## Initialize Tensorboard Logging Class 

In [343]:
mnist_net = Net().to(device)
mnist_net.load_state_dict(torch.load(model_path, map_location=torch.device(device)))  # load saved model 

mnist_net.eval()

Net(
  (conv1): Conv2d(1, 8, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=3, bias=True)
  (fc4): Linear(in_features=3, out_features=3, bias=True)
  (fc5): Linear(in_features=3, out_features=10, bias=True)
)

In [344]:
deep_features = DeepFeatures(model=mnist_net)

In [345]:
def filter_by_classes(data_loader, cl1, cl2):
    
    class1 = torch.tensor(cl1)
    class2 = torch.tensor(cl2)
    
    class1_images = []
    class2_images = []
    
    for images, labels in data_loader: 
        class1_images.extend(images[labels == class1])
        class2_images.extend(images[labels == class2])
        
    class1_images = torch.stack(class1_images)
    class2_images = torch.stack(class2_images)
        
    return class1_images, class2_images

In [346]:
def find_class_embeddings(deep_features, data_loader, cl1, cl2):
    images1, images2 = filter_by_classes(data_loader, cl1, cl2)
    
    e1 = deep_features.generate_embeddings(images1).clone().detach()
    e2 = deep_features.generate_embeddings(images2).clone().detach()
    
    model_if = IsolationForest(random_state=42)
    model_if.fit(e1.numpy())
    e1_anomaly = model_if.predict(e1)
    
    model_if.fit(e2.numpy())
    e2_anomaly = model_if.predict(e2)
    
    # keeping the inliers 
    e1 = e1[e1_anomaly==1]
    print('inliers for ' + str(cl1) + ' : ', e1.shape[0])
    e2 = e2[e2_anomaly==1]
    print('inliers for ' + str(cl2) + ' : ', e2.shape[0])
    
    ################## STEP 1: finding centroid of the 2 class embeddings  
    class1_mean = torch.mean(e1, dim=0)    # squish multiple rows to get mean 
    class2_mean = torch.mean(e2, dim=0)

    class1_normed = class1_mean/torch.norm(class1_mean)
    class2_normed = class2_mean/torch.norm(class2_mean)
    
    print('val of centroid of class ' + str(cl1) + ' : ', class1_normed)
    print('val of centroid of class ' + str(cl2)  + ' : ', class2_normed)
    print('cosine similarity : ', (torch.dot(class1_normed, class2_normed)/torch.linalg.norm(class1_normed)*torch.linalg.norm(class2_normed)))
    
    return class1_normed, class2_normed

In [347]:
x1, x2 = find_class_embeddings(deep_features, 
                               data_loader, 
                               merge_class1, 
                               merge_class2)

inliers for 2 :  890
inliers for 4 :  847
val of centroid of class 2 :  tensor([0.9771, 0.1258, 0.1719])
val of centroid of class 4 :  tensor([-0.4181, -0.5494,  0.7234])
cosine similarity :  tensor(-0.3532)


In [348]:
################## Step 3: calculate change of basis matrix
def change_of_basis(wm, d):
    ################## The first basis is d, how do we get the other 2 ?    

    new_wm = torch.eye(wm.shape[0])
    
    # replace first column of identity matrix with d 
    new_wm[:, 0] = d
    
    print('new weight matrix : ', new_wm)

    return new_wm

In [349]:
def perform_weight_surgery(mnist_net, x1, x2, save_model=True):
    
    ################## Step 2: calculating difference vector of the 2 classes 
    # calculate the difference between the 2 class vectors 
    d = x1 - x2  
    print('d : ', d)
#     d = d/torch.norm(d)
    print('normed d : ', d)
    weight_matrix = mnist_net.fc4.weight.clone().detach()  # weight matrix of the penultimate layer 
    print('weight matrix : \n', weight_matrix)
    
    ################## Step 3: calculate change of basis matrix 
    # perform a change of basis, to put d as the first basis 
    new_wm = change_of_basis(weight_matrix, d)
    
    # gram schmidt to calculate orthogonal basis vectors 
    def gram_schmidt(weight_matrix):
        ################## d is first basis vector
        new_basis_vectors = [weight_matrix[:, 0]]
        m, n = weight_matrix.shape

        for i in range(1, n):

            a_i = weight_matrix[:, i].clone()
            q_i = 0

            for j, q_j in enumerate(new_basis_vectors):
                q_i -= (torch.dot(q_j, a_i) * q_j)
            
            q_i += a_i
            q_i = q_i / torch.norm(q_i)
            
            new_basis_vectors.append(q_i)
            
        return new_basis_vectors
    
    ################## Step 4: using Gram-Schmidt to get orthogonal basis vectors 
#     new_basis_vectors = gram_schmidt(new_wm)    # 
    
#     U = [nbv.reshape(-1, 1) for nbv in new_basis_vectors]
#     U = torch.cat(U, axis=1)
    
    U, _ = torch.qr(new_wm)
    print('U matrix: ', U)
    print('result of U * U_transpose : ', torch.mm(U, U.T))
#     print('result of U * U_inv : ', U @ torch.inverse(U))
#     print('QR decomp :\n', Q)
    
    S = torch.eye(U.shape[0])
    S[0, 0] = 1e-5
#     S[0, 0] = 0.
#     print('matrix S :\n', S)
    
    # projection matrix 
    P_d = torch.mm(torch.mm(U, S), U.T)

    print('projection matrix : ', P_d)
    
    modified_weight_matrix = torch.mm(P_d, weight_matrix)
    
    print('modified weight matrix :\n', modified_weight_matrix)
    
    print('rank of the modified weights matrix : ', torch.linalg.matrix_rank(modified_weight_matrix))
    
    mnist_net.fc4.weight = torch.nn.Parameter(modified_weight_matrix)
    
    if save_model: 
        path = './modified_mnist_net.pth'
        torch.save(mnist_net.state_dict(), path)

In [350]:
perform_weight_surgery(mnist_net, x1, x2, save_model=True)

d :  tensor([ 1.3951,  0.6752, -0.5515])
normed d :  tensor([ 1.3951,  0.6752, -0.5515])
weight matrix : 
 tensor([[ 0.4307, -0.1483,  0.8380],
        [ 0.0444, -1.0082,  0.3290],
        [ 1.0885, -0.1497, -0.8214]])
new weight matrix :  tensor([[ 1.3951,  0.0000,  0.0000],
        [ 0.6752,  1.0000,  0.0000],
        [-0.5515,  0.0000,  1.0000]])
U matrix:  tensor([[-8.4802e-01,  3.8169e-01,  3.6765e-01],
        [-4.1044e-01, -9.1189e-01,  6.7842e-09],
        [ 3.3526e-01, -1.5090e-01,  9.2996e-01]])
result of U * U_transpose :  tensor([[ 1.0000e+00, -8.6913e-08,  8.8007e-08],
        [-8.6913e-08,  1.0000e+00,  2.1210e-08],
        [ 8.8007e-08,  2.1210e-08,  1.0000e+00]])
projection matrix :  tensor([[ 0.2809, -0.3481,  0.2843],
        [-0.3481,  0.8315,  0.1376],
        [ 0.2843,  0.1376,  0.8876]])
modified weight matrix :
 tensor([[ 0.4150,  0.2667, -0.1127],
        [ 0.0368, -0.8074, -0.1311],
        [ 1.0948, -0.3137, -0.4456]])
rank of the modified weights matrix :  te

In [351]:
original_mnist_net = Net().to(device)
original_mnist_net.load_state_dict(torch.load('./mnist_net.pth', map_location=torch.device(device)))  # load saved model 

original_mnist_net.eval()

original_deep_features = DeepFeatures(model=original_mnist_net)

In [216]:
_, _ = find_class_embeddings(original_deep_features, data_loader, merge_class1, merge_class2)

inliers for 2 :  899
inliers for 4 :  854
val of centroid of class 2 :  tensor([0.9772, 0.1251, 0.1716])
val of centroid of class 4 :  tensor([-0.4172, -0.5502,  0.7233])
cosine similarity :  tensor(-0.3525)


In [352]:
_, _ = find_class_embeddings(deep_features, data_loader, merge_class1, merge_class2)

inliers for 2 :  879
inliers for 4 :  862
val of centroid of class 2 :  tensor([ 0.2439, -0.4704,  0.8481])
val of centroid of class 4 :  tensor([-0.0767, -0.5391,  0.8388])
cosine similarity :  tensor(0.9462)


## View Unmodified Embeddings through Plotly

In [353]:
def remove_outliers(embeds, labels):
    unique_labels = set(labels.numpy())
    
    model_if = IsolationForest(random_state=42)
    
    new_embeds, new_labels = [], []
    
    for label_x in unique_labels:
        embed_x = embeds[labels==label_x]
        
        model_if.fit(embed_x)
        embedx_anomaly = model_if.predict(embed_x)
        
        embed_x = embed_x[embedx_anomaly == 1]   # keeping inliers 
        labels_x = [label_x] * embed_x.shape[0]
        
        new_embeds.extend(embed_x)
        new_labels.extend(labels_x)
        
    return np.stack(new_embeds), np.stack(new_labels)

In [354]:
batch_images, batch_labels = next(iter(data_loader))

embeds = original_deep_features.generate_embeddings(batch_images.to(device), normalize=True)
embeds = embeds.detach().numpy()

# new_embeds, new_labels = remove_outliers(embeds, batch_labels)

In [355]:
df = pd.DataFrame({'x': embeds[:, 0], 
                   'y': embeds[:, 1], 
                   'z': embeds[:, 2], 
                   'label': batch_labels.numpy()})

fig = px.scatter_3d(df, x='x', y='y', z='z',
                      color='label', text='label')
fig.update_traces(marker_size = 5)
fig.show()

## Write Modified Embeddings through Plotly

In [356]:
batch_images, batch_labels = next(iter(data_loader))

embeds = deep_features.generate_embeddings(batch_images.to(device), normalize=True)
embeds = embeds.detach().numpy()

# new_embeds, new_labels = remove_outliers(embeds, batch_labels)

In [357]:
def project_to_2D_unit_circle(embeds):
    
    # Find the normal vector of the plane
    u, s, vh = np.linalg.svd(embeds)
    normal = vh[-1]    # the last eigenvector is the one that doesn't vary, i.e. 
                       # normal to the plane where the points lie 

    # Project the points onto the plane
    points_proj = embeds - (embeds @ np.outer(normal, normal))  # embeds minus projection of embeds onto normal
                                                                # would give us embeds that lie on the 2D plane,
                                                                # which is perpendicular to the normal 

    # normalize to unit circle 
    points_norm = np.linalg.norm(points_proj, axis=1)
    points_proj /= points_norm[:, np.newaxis]         # using broadcasting to normalize 
    
    return points_proj

In [358]:
projected_points = project_to_2D_unit_circle(embeds)
df = pd.DataFrame({'x': projected_points[:, 0], 
                   'y': projected_points[:, 1], 
                   'z': projected_points[:, 2], 
                   'label': batch_labels.numpy()})

fig = px.scatter_3d(df, x='x', y='y', z='z',
                      color='label', text='label')
fig.update_traces(marker_size = 5)
fig.show()

## Accuracy over test images

In [359]:
def accuracy_over_test_images(mnist_net, tag='original'):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device); labels = labels.to(device)
            # calculate outputs by running images through the network
            outputs = mnist_net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the ' +  tag +  ' network on the '+ str(total) +' test images: '+ str(100 * correct // total)+ '%')

In [360]:
accuracy_over_test_images(mnist_net, tag='modified')

Accuracy of the modified network on the 10000 test images: 66%


In [79]:
# accuracy_over_test_images(original_mnist_net)
# Accuracy of the original network on the 10000 test images: 97%

Accuracy of the original network on the 10000 test images: 97%
